In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

from collections import namedtuple
from IPython.display import clear_output
from tqdm.notebook import tqdm
from numpy.typing import NDArray

from scipy.optimize import minimize
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit.circuit import Parameter, Instruction
from qiskit.circuit.library import RXGate, RYGate, RZGate, CXGate, CZGate, IGate
from qiskit_aer import AerSimulator

In [3]:
from qml.model.gate import get_gateset, GateInfo, Gate

In [53]:
class Unit:
    
    VALUE_MAX = 2 * np.pi
    
    def __init__(
            self,
            num_qubits: int,
            name: str,
            gates: list[Gate],
            params: list[Parameter],
            qubits: list[int],
            values: list[float] | NDArray
    ):
        self._num_qubits = num_qubits
        self.name = name
        self._gates = gates
        self._params = params
        self._qubits = qubits
        self._param_values = np.asarray(values) % self.VALUE_MAX
        
    def feed_dict(self, values=None) -> dict[str, float]:
        if values is None:
            values = self.values
        if not hasattr(values, "__len__"):
            values = [values]
        assert len(values) == len(self.parameters), f"Length of values {len(values)} must be equal to number of parameters {len(self.values)}"
        
        feed_dict = dict()
        for param, value in zip(self._params, values):
            feed_dict |= {
                param.name: value
            }
        return feed_dict
    
    def apply_to_circuit(self, qc: QuantumCircuit) -> QuantumCircuit:
        for gate in self._gates:
            if gate.multi_qubit:
                qargs = [
                    qc.qubits[gate.qubit]
                ]
            
            
    @staticmethod
    def generate_random_unit(
            num_qubit: int,
            num_gate: int,
            name: str = None,
            gateset: dict[GateInfo] = None,
    ):
        if gateset is None:
            gateset = get_gateset(num_qubit)
        
        # select gates at random
        gate_names_on_set = list(gateset.keys())
        gate_names = np.random.choice(gate_names_on_set, size=num_gate, replace=True)
        gate_infos = [gateset[gate_name] for gate_name in gate_names]
        
        # select qubits to apply
        qubits = np.random.randint(0, num_qubit, size=num_gate)
        
        # build instance of gates and parameters
        gates = []
        params = []
        for gate_info, qubit in zip(gate_infos, qubits):
            if not gate_info.trainable:
                gates.append(Gate.new_with_info(gate_info, qubit))
                continue
            
            pname = f"param_{len(params)}"
            if name is not None:
                pname = name + "_" + pname
            param = Parameter(pname)
            params.append(param)
            gates.append(Gate.new_with_info(gate_info, qubit, param))
                
        # initialize parameter values
        values = np.zeros(len(params))
        
        return Unit(num_qubit, name, gates, params, qubits, values)
    
    @property
    def num_qubits(self):
        return self._num_qubits
    
    @property
    def nq(self):
        return self.num_qubits
    
    @property
    def values(self):
        return self._param_values.copy()
    
    @values.setter
    def values(self, values):
        assert len(values) == len(self.values), f"Length of values {len(values)} must be equal to number of parameters {len(self.values)}"
        values = np.asarray(values)
        values = values % self.VALUE_MAX
        self._param_values = values
        
    @property
    def parameters(self):
        return self.values
    
    @parameters.setter
    def parameters(self, values):
        self.values = values
    
    @property
    def gates(self):
        return [gate for gate in self._gates]
    
    @property
    def parameter_instances(self):
        return self._params
    
    @property
    def num_param(self):
        return len(self._params)

In [54]:
unit = Unit.generate_random_unit(3, 3, name="demo")
unit.apply_to_circuit(None)
# unit.gates

<__main__.Gate object at 0x13e0dd7f0> 1
<__main__.Gate object at 0x13e0dd220> 0
<__main__.Gate object at 0x13e0dd2e0> 2
